####  Extending P0 with Bitwise Set Operations (WASM)

In [1]:
def runwasm(wasmfile):
    from IPython.display import display, Javascript
    display(Javascript("""
    const params = {
        P0lib: {
            write: i => element.append(i + ' '),
            writeln: () => element.append(document.createElement('br')),
            read: () => window.prompt()
        }
    }
    
    var wasmByteString = \"""" + str(open(wasmfile, "rb").read()) + """\"; // pass the wasm file to JavaScript as byte string
    wasmByteString = wasmByteString.substring(2, wasmByteString.length - 1); // remove the byte literals b'...'
    const wasmArrayBuffer = new Uint8Array(wasmByteString.length); // convert the binary string to ArrayBuffer
    for (let i = 0; i < wasmByteString.length; i++)
      wasmArrayBuffer[i] = wasmByteString.charCodeAt(i);
    
    WebAssembly.compile(wasmArrayBuffer.buffer) // compile (sharable) code
        .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
        // .then(instance => instance.exports.program()); // run the main program; not needed if a start function is specified
     """))

In [2]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [3]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [4]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

Extend P0 with two binary operators, `∉` (Unicode U+2209, not an element of) and `∖` (Unicode U+2216, set minus), defined by:
* `i ∉ s ≡ ¬(i ∈ s)`, where `i` is an integer and `s` is a set 
* `s ∖ t = s ∩ ∁t`, where `s`, `t` are sets

The following P0 program illustrates the use of these operators; you can use it to test your implementation:

In [5]:
compileString("""
const N = 32
const R = 5 // ⌊√N⌋
type S = set [0 .. N - 1]
procedure eratosthenes() → (p: S)
    var i, j: integer
        p := ∁ {} // set of all integers 
        i := 2;
        while i ≤ R do
            if i ∈ p then
                j := i × i
                while j < N do
                    p, j := p ∖ {j}, j + i
            i := i + 1
program primes
    var p: S
    var j: integer
        p ← eratosthenes()
        j := 2 // print all primes up to N
        while j < N do
            if j ∈ p then write(j)
            j := j + 1
        writeln(); j := 2 // print all non-primes up to N
        while j < N do
            if j ∉ p then write(j)
            j := j + 1
""", 'primes.wat', target = 'wat')

In [6]:
!wat2wasm primes.wat

In [7]:
runpywasm("primes.wasm")

2 3 5 7 11 13 17 19 23 29 31 
4 6 8 9 10 12 14 15 16 18 20 21 22 24 25 26 27 28 30 

The output is supposed to be:
```
2 3 5 7 11 13 17 19 23 29 31 
4 6 8 9 10 12 14 15 16 18 20 21 22 24 25 26 27 28 30
```

##### Part A (Extending the Scanner) [4 points]

- In `SC.ipynb`, introduce new integer constants `NOTELEMENT` and `DIFFERENCE` for `∉` and `∖`.
- Extend the production of `symbol` in the text cell above `getSym()` to include  `∉` and `∖`.
- Extend `getSym()` to return `NOTELEMENT` and `DIFFERENCE` when recognizing  `∉` and `∖`.

The cell below serves only for grading and testing.

In [8]:
import SC

def scanString(src):
    SC.init(src); syms = [(SC.sym, SC.val)]
    while SC.sym != SC.EOF:
        SC.getSym()
        syms.append((SC.sym, SC.val))
    return syms

assert scanString('5 ∉ {3}') == [(SC.NUMBER, 5), (SC.NOTELEMENT, 5), (SC.LBRACE, 5), 
                                 (SC.NUMBER, 3), (SC.RBRACE, 3), (SC.EOF, 3)]
assert scanString('s ∖ t') == [(SC.IDENT, 's'), (SC.DIFFERENCE, 's'), (SC.IDENT, 't'), (SC.EOF, 't')]
    

##### Part B (Extending the Parser) [8 points]

1. In `P0.ipynb`, extend the imports with `NOTELEMENT` and `DIFFERENCE`.
2. Extend the grammar at two places, in the section "The P0 Grammar" and before the parsing procedures for each nonterminal: `∖` has to bind as tight as `∩` and `∉` has to bind as tight as `∈`. That is, `a ∉ b ∖ c ∪ d` is parsed like `a ∉ ((b ∖ c) ∪ d)`.
3. If the first sets need to be modified, modify them.
4. Extend the parsing procedures accordingly. Add type-checking: when parsing `a ∉ b`, if `a` is not an integer, the error `bad type` should be generated; if `b` is not a set, the error `set expected` should be generated. When parsing `(a ∖ b)`, the error `bad type` should be generated if not both `a` and `b` are sets.
5. In `CGast.ipynb`, extend the imports with `NOTELEMENT` and `DIFFERENCE`.
6. Extend method `__str(self)__` of class `BinaryOp` such that `∉` is returned for `NOTELEMENT` and `∖` is returned for `DIFFERENCE`.

The cell below serves only for grading and testing.

In [9]:
assert compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    c := (2 ∈ a) or (2 ∉ a)
    a := ∁ a ∩ b ∖ a ∪ b
""", target='ast') == """\
seq
  :=
    Var(name = c, lev = 1, tp = <class 'ST.Bool'>)
    or
      ∈
        Const(name = , tp = <class 'ST.Int'>, val = 2)
        Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
      ∉
        Const(name = , tp = <class 'ST.Int'>, val = 2)
        Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
  :=
    Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
    ∪
      ∖
        ∩
          ∁
            Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
          Var(name = b, lev = 1, tp = Set(lower = 1, length = 5))
        Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
      Var(name = b, lev = 1, tp = Set(lower = 1, length = 5))"""

The following programs should all produce error messages:

In [10]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    c := a ∉ b
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 5 pos 14 bad type"

In [11]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    c := 2 ∉ c
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 5 pos 14 set expected"

In [12]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    a := c ∖ b
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 5 pos 14 bad type"

In [13]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
    a := b ∖ 3
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 4 pos 14 bad type"

##### Part C (Extending the Code Generator) [8 points]

In `CGwat.ipynb`, extend `genUnaryOp` and `genBinaryOp` to generate code for `∉` and `∖`.

The cell below serves only for grading and testing. *Hint:* when generating code for `a ∖ b` in `genBinaryOp`, load first `b` on the stack, complement it, then load `a` on the stack and bitwise-and the top two elements on the stack.

In [14]:
import nbimporter; nbimporter.options["only_defs"] = False
import P0

compileString("""
program bitsets
  var a, b: set [1 .. 11]
  var i: integer
    i, a := 1, {3, 5, 7}
    if i ∉ a then a := a ∖ {7, 9}
    while i < 12 do
      if i ∈ a then write(i)
      i := i + 1
""", 'bitsets.wat', target = 'wat')

assert open('bitsets.wat').read() == """\
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $a i32)
(local $b i32)
(local $i i32)
(local $0 i32)
i32.const 1
i32.const 3
local.set $0
i32.const 1
local.get $0
i32.shl
i32.const 5
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
i32.const 7
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
local.set $a
local.set $i
local.get $i
local.set $0
i32.const 1
local.get $0
i32.shl
local.get $a
i32.and
i32.eqz
if
i32.const 7
local.set $0
i32.const 1
local.get $0
i32.shl
i32.const 9
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
i32.const 0xffe
i32.xor
local.get $a
i32.and
local.set $a
end
loop
local.get $i
i32.const 12
i32.lt_s
if
local.get $i
local.set $0
i32.const 1
local.get $0
i32.shl
local.get $a
i32.and
if
local.get $i
call $write
end
local.get $i
i32.const 1
i32.add
local.set $i
br 1
end
end
)
(memory 1)
(start $program)
)"""


The first if-statement should translate to:
```
local.get $i
local.set $0
i32.const 1
local.get $0
i32.shl
local.get $a
i32.and
i32.eqz
if
i32.const 7
local.set $0
i32.const 1
local.get $0
i32.shl
i32.const 9
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
i32.const 0xffe
i32.xor
local.get $a
i32.and
local.set $a
end
```

Running the program should print `3 5`:

In [15]:
!wat2wasm bitsets.wat

In [16]:
runpywasm("bitsets.wasm")

3 5 

##### Part D (Evaluating the Implementation) [4 points].

The task is to compare the efficiency of 4 implementations of the Sieve of Eratosthenes with sets:
- P0 using pywasm, the Python interpreter of WebAssembly,
- P0 using the JavaScript host of WebAssembly in your web browser,
- Python using the standard CPython implementation,
- Java using the standard Oracle JVM.

The P0, Python, and Java implementations are already provided. To keep it simple, the [Jupyter time cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) is used rather than printing the start and end time within the programs. That command measures the *wall clock time*. This means that the time for loading and compilation is included in the measured time. We, therefore, aim at execution times above 1 second to make the time for loading and compiling negligible. For this, the Sieve of Eratosthenes is repeatedly run. The times will vary with each run. Longer times typically result from an unrelated CPU load and should be disregarded. The best is to run the code at times of the day when there is as little interference as possible. Vary the number of repetitions. Document your observations and explain!  

Running P0 using the JavaScript host of WebAssembly in my web browser was the quickest to run staying around <6 ms. The python interpreter of WebAssembly was the second fastest as it was consistently around just under 900ms. The CPython implementation was quite abit slower, usually hovering around the 5 second mark. Lastly, the Java implementation using the standard Oracle JVM was the slowest with run times closer to a minute.

Therefore, based off of these findings, any WebAssembly (with the JavaScript host more than the Python interpreter) would be much more efficient than compiling P0 through Python or Java as these languages seem to struggle to efficiently evaluate compileString and take several seconds (or minutes in Java's case) to do so.

In [17]:
compileString("""
const N = 32
const R = 5 // ⌊√N⌋
type S = set [2 .. 31]
procedure eratosthenes() → (p: S)
    var i, j: integer
        p := ∁ {} // set of all integers 
        i := 2;
        while i ≤ R do
            if i ∈ p then
                j := i × i
                while j < N do
                    p, j := p ∖ {j}, j + i
            i := i + 1
program repeatprimes
    var p: S
    var j: integer
        j := 0
        while j < 1000 do
            p ← eratosthenes(); j := j + 1
""", 'repeatprimes.wat', target = 'wat')

In [18]:
!wat2wasm repeatprimes.wat

In [19]:
time runpywasm("repeatprimes.wasm")

CPU times: user 893 ms, sys: 411 µs, total: 893 ms
Wall time: 899 ms


In [20]:
time runwasm("repeatprimes.wasm")

<IPython.core.display.Javascript object>

CPU times: user 4.24 ms, sys: 1.14 ms, total: 5.38 ms
Wall time: 4.89 ms


In [21]:
time runwasmer("repeatprimes.wasm")

CPU times: user 17.5 ms, sys: 12.1 ms, total: 29.6 ms
Wall time: 8.52 ms


In [22]:
def eratosthenes():
    N, R = 32, 5
    p = {i for i in range(2, N)}
    for i in range(2, R):
        if i in p:
            for j in range(i * i, N, i): p -= {j}
    return p

def repeatprimes():
    for _ in range(1000000): eratosthenes()

In [23]:
time repeatprimes()

CPU times: user 5.23 s, sys: 5.22 ms, total: 5.23 s
Wall time: 5.25 s


In [24]:
%%writefile repeatprimes.java
import java.util.*;
class RepeatPrimes {
    static final int N = 32;
    static final int R = 5;
    static Set<Integer> eratosthenes() {
        Set<Integer> p = new HashSet<Integer>();
        for (int i = 2; i < N; i++) p.add(i);
        for (int i = 2; i < R; i++)
            if (p.contains(i))
                for (int j = i * i; j < N; j = j + i)
                    p.remove(j);
        return p;
    }
    public static void main(String[] args) {
        for (int i = 0; i < 100000000; i++) eratosthenes();
    }
}

Overwriting repeatprimes.java


In [25]:
!javac repeatprimes.java

In [26]:
time !java RepeatPrimes

CPU times: user 619 ms, sys: 147 ms, total: 766 ms
Wall time: 58.8 s
